In [323]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import time


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from helpers import *
from helpers_create_data import *
from implementations import *

### LOADING THE CSV DATA INTO ARRAYS

In [28]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data(".", sub_sample=False)

#### FEATURE CHOICES :
From the dataset, we filter the features in 3 different ways :

1. Using (almost) all the features

2. Choosing 19 features (taken mostly from https://medium.com/@alexteboul17/building-predictive-models-for-heart-disease-using-the-2015-behavioral-risk-factor-surveillance-b786368021ab)

3. From the 19 features, we pick the 10 best ones using the best correlations towards the y variable

### DATA PREPROCESSING

#### 1. All features

In [380]:
# we drop the rows that has a NaN percentage of {threshold} because we assume that they don't offer much information
X_tr_all, Y_tr_all = drop_rows_with_nan(x_train, y_train, threshold=0.4)

# we process the dataset by replacing the remaining NaNs by column with the mode of the feature column that have less than 10 unique values and by its 
# mean if the feature column has more than 10 unique values. Also we remove the columns that have extremely low variance as this column 
# doesn't offer any information and we might encouter numerical issues when standardizing.
X_tr_all, X_test_all = process_datasets(X_tr_all, x_test, unique_values_thresh=10)

# we replace the the -1s in the y column by 0s
Y_tr_all[Y_tr_all == -1] = 0

# We standardize the datasets in order to give us better numerical results
X_tr_all, mean_x_tr_all, std_x_tr_all = standardize(X_tr_all)
X_test_all,_,_ = standardize(X_test_all, mean_x_tr_all, std_x_tr_all)

# We now balance the data to a slightly more balanced ratio of 0s and 1s
X_tr_all, Y_tr_all = undersampling_oversampling(X_tr_all, Y_tr_all, ratio_majority=0.5, ratio_majority_to_minority=2)

# We add a column of ones (bias term) to the dataset
X_tr_all = np.c_[np.ones((X_tr_all.shape[0], 1)), X_tr_all]
X_test_all = np.c_[np.ones((X_test_all.shape[0], 1)), X_test_all]

#### 2. 19 Best Features

In [382]:
# We had to create the make_data function (-> helpers_create_data) because we manually preprocess 
# each feature in this case and do all the necessary further preprocessing within the function 
X_tr_19, Y_tr_19, X_test_19 = make_data('./x_train.csv', './x_test.csv', x_train, x_test, y_train, replace=True)

# Same as for all features, we re-balance the dataset 
X_tr_19, Y_tr_19 = undersampling_oversampling(X_tr_19, Y_tr_19, ratio_majority=1, ratio_majority_to_minority=2)

# We add a column of ones (bias term) before training
X_tr_19 = np.c_[np.ones((X_tr_19.shape[0], 1)), X_tr_19]
X_test_19 = np.c_[np.ones((X_test_19.shape[0], 1)), X_test_19]

#### 3. 10 Best Features

In [401]:
# We compute the correlations of all the feature variables with the output variable
correlations = np.zeros(X_train_balanced.shape[1])
for i in range(len(correlations)):
    correlations[i] = np.abs(np.corrcoef(X_train_balanced[:,i], Y_train_balanced[:,0])[0,1])

# We take the 10 biggest ones
best_10_idx = np.argsort(correlations)[:-11:-1]

# We use those 10 features for the dataset
X_tr_10 = X_tr_19[:,best_10_idx]
X_test_10 = X_test_19[:,best_10_idx]
Y_tr_10 = Y_tr_19

# We add a column of ones (bias term) before training
X_tr_10 = np.c_[np.ones((X_tr_10.shape[0], 1)), X_tr_10]
X_test_10 = np.c_[np.ones((X_test_10.shape[0], 1)), X_test_10]

### MODELS TRAINING

#### 1. All features
(takes a while to run...)

In [394]:
max_iter = 10000

k_fold = 5

lambdas = np.logspace(-7,0,k_fold)
gammas = np.logspace(-7,0,k_fold)

accuracies_all = np.zeros((k_fold,k_fold))
f1_scores_all = np.zeros((k_fold,k_fold))

ws_all = []
losses_all = []

w_reg_all = np.zeros(X_tr_all.shape[1])

for i, lambda_ in enumerate(lambdas):
    for j, gamma in enumerate(gammas):
        # each iteration we pick a new training and a new val set
        X_train, X_val, Y_train, Y_val = split_train_val(X_tr_all, Y_tr_all, k_fold, j)
        
        #Reshape form (#points,1) to (#points,) in order to use the implemented logistic regression function
        Y_train = Y_train.reshape(-1)
        Y_val = Y_val.reshape(-1)

        #Train model (-> our train set) using stocha logistic regression
        w, loss = reg_logistic_regression_stoch(Y_train, X_train, lambda_, w_reg_all, max_iter, gamma)
        Y_pred = prediction(X_val, w)

        accuracies_all[i,j] = compute_accuracy(Y_val, Y_pred)
        f1_scores_all[i,j] = f1(Y_pred, Y_val)
        ws_all.append(w)
        losses_all.append(loss)
        
best_idx_f1_all = np.unravel_index(np.argmax(f1_scores_all), f1_scores_all.shape)
best_idx_acc_all = np.unravel_index(np.argmax(accuracies_all), accuracies_all.shape)
# We pick our w according to the optimal f1_score
best_w_all = ws_all[np.argmax(f1_scores_all)]
print(f'When using all features, the best index for accuracy is {best_idx_acc_all} and the best one for f1 score is {best_idx_f1_all} on the val set.')
print(f'\nWhen using all features, the best accuracy is {accuracies_all[best_idx_acc_all]} and the best f1 score is {f1_scores_all[best_idx_acc_all]} on the val set.')

When using all features, the best index for accuracy is (0, 2) and the best one for f1 score is (2, 2).

When using all features, the best accuracy is 73.20987654320987 and the best f1 score is 0.6444080294961082.


#### 2. 19 Features
takes a looooong time... (please don't rerun)

In [399]:
max_iter = 10000

k_fold = 5

lambdas = np.logspace(-7,0,k_fold)
gammas = np.logspace(-7,0,k_fold)

accuracies_19 = np.zeros((k_fold,k_fold))
f1_scores_19 = np.zeros((k_fold,k_fold))

ws_19 = []
losses_19 = []

w_reg_19 = np.zeros(X_tr_19.shape[1])

for i, lambda_ in enumerate(lambdas):
    for j, gamma in enumerate(gammas):
        # each iteration we pick a new training and a new val set
        X_train, X_val, Y_train, Y_val = split_train_val(X_tr_19, Y_tr_19, k_fold, j)
        
        #Reshape form (#points,1) to (#points,) in order to use the implemented logistic regression function
        Y_train = Y_train.reshape(-1)
        Y_val = Y_val.reshape(-1)

        #Train model (-> our train set) using stocha logistic regression
        w, loss = reg_logistic_regression_stoch(Y_train, X_train, lambda_, w_reg_19, max_iter, gamma)
        Y_pred = prediction(X_val, w)

        accuracies_19[i,j] = compute_accuracy(Y_val, Y_pred)
        f1_scores_19[i,j] = f1(Y_pred, Y_val)
        ws_19.append(w)
        losses_19.append(loss)
        
best_idx_f1_19 = np.unravel_index(np.argmax(f1_scores_19), f1_scores_19.shape)
best_idx_acc_19 = np.unravel_index(np.argmax(accuracies_19), accuracies_19.shape)
# We pick our w according to the optimal f1_score
best_w_19 = ws_19[np.argmax(f1_scores_19)]
print(f'When using 19 features, the best index for accuracy is {best_idx_acc_19} and the best one for f1 score is {best_idx_f1_19} on the val set.')
print(f'\nWhen using 19 features, the best accuracy is {accuracies_19[best_idx_acc_19]} and the best f1 score is {f1_scores_19[best_idx_acc_19]} on the val set.')

When using 19 features, the best index for accuracy is (0, 3) and the best one for f1 score is (0, 2).

When using 19 features, the best accuracy is 77.76886393011544 and the best f1 score is 0.6406598948202579.


#### 3. 10 Features

In [403]:
max_iter = 10000

k_fold = 5

lambdas = np.logspace(-7,0,k_fold)
gammas = np.logspace(-7,0,k_fold)

accuracies_10 = np.zeros((k_fold,k_fold))
f1_scores_10 = np.zeros((k_fold,k_fold))

ws_10 = []
losses_10 = []

w_reg_10 = np.zeros(X_tr_10.shape[1])

for i, lambda_ in enumerate(lambdas):
    for j, gamma in enumerate(gammas):
        # each iteration we pick a new training and a new val set
        X_train, X_val, Y_train, Y_val = split_train_val(X_tr_10, Y_tr_10, k_fold, j)
        
        #Reshape form (#points,1) to (#points,) in order to use the implemented logistic regression function
        Y_train = Y_train.reshape(-1)
        Y_val = Y_val.reshape(-1)

        #Train model (-> our train set) using stocha logistic regression
        w, loss = reg_logistic_regression_stoch(Y_train, X_train, lambda_, w_reg_10, max_iter, gamma)
        Y_pred = prediction(X_val, w)

        accuracies_10[i,j] = compute_accuracy(Y_val, Y_pred)
        f1_scores_10[i,j] = f1(Y_pred, Y_val)
        ws_10.append(w)
        losses_10.append(loss)
        
best_idx_f1_10 = np.unravel_index(np.argmax(f1_scores_10), f1_scores_10.shape)
best_idx_acc_10 = np.unravel_index(np.argmax(accuracies_10), accuracies_10.shape)
# We pick our w according to the optimal f1_score
best_w_10 = ws_10[np.argmax(f1_scores_10)]
print(f'When using 10 features, the best index for accuracy is {best_idx_acc_10} and the best one for f1 score is {best_idx_f1_10} on the val set.')
print(f'\nWhen using 10 features, the best accuracy is {accuracies_10[best_idx_acc_10]} and the best f1 score is {f1_scores_10[best_idx_acc_10]} on the val set.')

When using 10 features, the best index for accuracy is (0, 3) and the best one for f1 score is (0, 4) on the val set.

When using 10 features, the best accuracy is 73.58715514551857 and the best f1 score is 0.5588372136303575 on the val set.


### GENERATING THE PREDICTIONS

#### 1. All features

In [406]:
Y_test_all = prediction(X_test_all,best_w_all)
Y_test_all[Y_test_all == 0] = -1
create_csv_submission(test_ids, Y_test_all, "Y_test_all")

#### 2. 19 features

In [409]:
Y_test_19 = prediction(X_test_19, best_w_19)
Y_test_19[Y_test_19 == 0] = -1
create_csv_submission(test_ids, Y_test_19, "Y_test_19")

#### 3. 10 features

In [412]:
Y_test_10 = prediction(X_test_10, best_w_10)
Y_test_10[Y_test_10 == 0] = -1
create_csv_submission(test_ids, Y_test_10, "Y_test_10")